In [2]:
import pandas as pd

/var/folders/k_/x2cc2f4901v6lyz_t7w3w3h40000gn/T/ipykernel_34546/4080736814.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
data = pd.read_csv('Key_Dynamics.csv')

In [4]:
data.drop(["sessionIndex", "rep"], axis=1, inplace=True)

In [5]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
import numpy as np

ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(),[0])],remainder='passthrough')
unique_cols = data['subject'].nunique()

data = np.array(ct.fit_transform(data))
X = data[:,unique_cols:]
y = data[:,:unique_cols]
print(y.shape)

(20400, 51)


In [13]:
X.shape

(20400, 31)

In [6]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 1)

In [7]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

with open("scale.pkl", "wb") as f:
    import pickle
    pickle.dump(sc, f)

In [8]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential()
model.add(Dense(units=128, activation='relu'))
model.add(Dense(units=64, activation='relu'))
model.add(Dense(units=unique_cols, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.fit(X_train, y_train, epochs=10, batch_size=32)

/Users/achintya/Achintya/Competitions/Kochi/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


Epoch 1/10


2024-02-04 09:00:19.716500: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M3
2024-02-04 09:00:19.716517: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2024-02-04 09:00:19.716523: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
2024-02-04 09:00:19.716729: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-02-04 09:00:19.717152: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


  8/510 [..............................] - ETA: 3s - loss: 4.0830 - accuracy: 0.0273  

2024-02-04 09:00:20.080802: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


510/510 [==============================] - 4s 7ms/step - loss: 1.9484 - accuracy: 0.5639
Epoch 2/10
510/510 [==============================] - 5s 10ms/step - loss: 1.3343 - accuracy: 0.7174
Epoch 3/10
510/510 [==============================] - 5s 9ms/step - loss: 1.1390 - accuracy: 0.7563
Epoch 4/10
510/510 [==============================] - 4s 8ms/step - loss: 1.0584 - accuracy: 0.7652
Epoch 5/10
510/510 [==============================] - 4s 8ms/step - loss: 1.0193 - accuracy: 0.7728
Epoch 6/10
510/510 [==============================] - 4s 7ms/step - loss: 0.9348 - accuracy: 0.7950
Epoch 7/10
510/510 [==============================] - 4s 8ms/step - loss: 0.9166 - accuracy: 0.7943
Epoch 8/10
510/510 [==============================] - 4s 8ms/step - loss: 0.9201 - accuracy: 0.7948
Epoch 9/10
510/510 [==============================] - 4s 8ms/step - loss: 0.8879 - accuracy: 0.8032
Epoch 10/10
510/510 [==============================] - 4s 7ms/step - loss: 0.8335 - accuracy: 0.8151


In [9]:
model.evaluate(X_test, y_test)

128/128 [==============================] - 1s 7ms/step - loss: 0.8515 - accuracy: 0.8020


[0.8515316247940063, 0.8019607663154602]

In [10]:
model.save('key_dynamics_model.h5')

/Users/achintya/Achintya/Competitions/Kochi/.venv/lib/python3.9/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [11]:
row_index = 94
output_row = X_test[row_index]
output_row = output_row.reshape(1, -1)
prediction = model.predict(output_row)
print(np.argmax(prediction))

1/1 [==============================] - 0s 187ms/step
4


In [12]:
output_row.shape

(1, 31)